In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf
from vahun.tools import Timer
from vahun.corpus import Corpus
from vahun.tools import explog
from vahun.variational_autoencoder import Variational_autoencoder
from vahun.tools import show_performance

encode=180
dictsize=2000000
popsize=1


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction=1
corp_path='/mnt/permanent/Language/Hungarian/Corp/Webkorpusz/webkorpusz.wpl'
corp=Corpus(corpus_path=corp_path,
            needed_corpus=["hun_lower_unique"],
            language="Hun",size=dictsize,encoding_len=10)
database=corp.hun_lower_unique
all_features=corp.featurize_data_charlevel_onehot(database)
train=all_features[0:int(len(all_features)*0.8)]
test=all_features[int(len(all_features)*0.8):len(all_features)]
x_train = train.reshape((len(train), np.prod(train.shape[1:])))
x_test = test.reshape((len(test), np.prod(test.shape[1:])))
print(x_train.shape)

logger=explog(encoder_type="variational_unique_"+str(encode),
              encoding_dim=encode,feature_len=10,
              lang="Hun",unique_words=len(set(database)),
              name="variational_unique_"+str(encode),population_size=popsize,
              words=len(corp.hun_lower_unique))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

Reading file, speed:  1871210.7828178732  words/s
Reading file, speed:  2191347.053076493  words/s
Corpus initalized, fields: ['hun_lower_unique'] 
Unique words:  214708
(81526, 380)


In [2]:
encoder=Variational_autoencoder(logger=logger,tf_session=sess,
                         inputdim=380,
                         encoding_size=180,
                         optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                         nonlinear=tf.sigmoid)
encoder.train(x_train,x_test,512,80)

In [3]:
stds=show_performance(encoder,x_test,corp,100,printer=True)

 feleséget 	  feleséget 	 0
kosarakban 	 kosarakban 	 0
 budavidék 	  budavidék 	 0
  alapozót 	   alapozót 	 0
  húzásnál 	   hízásnál 	 1
    méhben 	     méhben 	 0
 megtudhat 	  megtudhat 	 0
    műsört 	     mcsurt 	 2
   zealand 	    zealand 	 0
   ételsor 	    ételsor 	 0
  megfűzöm 	   megöözöm 	 2
  tagadója 	   tagadója 	 0
megaláznak 	 megaláznak 	 0
  gépeivel 	   gépeivel 	 0
elnököknek 	 elnököknek 	 0
 éjfeketén 	  épfeketén 	 1
 meghajtva 	  meghajtva 	 0
 ébresszék 	  ébresszék 	 0
  megadást 	   megadást 	 0
  mobilban 	   mobilban 	 0
társulatot 	 társulatot 	 0
presztízsű 	 éresztízsz 	 2
fekvésének 	 fekvésének 	 0
   nevéből 	    nevéből 	 0
  zuhanunk 	   zuhanunk 	 0
  vonjátok 	   vonjátok 	 0
 csináltuk 	  csináltuk 	 0
     idegi 	      idegi 	 0
 behajtjuk 	  behajtjuk 	 0
 játékaiba 	  vátékaiba 	 1
    bitkom 	     bitkom 	 0
  newsweek 	   necsreek 	 2
megfejelve 	 megfejelve 	 0
elnökökkel 	 elnökökkel 	 0
    ujjaik 	     ujjaik 	 0
      élne 	       é

In [4]:
stds=show_performance(encoder,["e","eh","kutya","aytuk","macska","árvíztűrő","fúró","kacsa","a","és"],corp,printer=True)

         e 	         ke 	 1
        eh 	         ee 	 1
     kutya 	      kutea 	 1
     aytuk 	      aytuk 	 0
    macska 	     macska 	 0
 árvíztűrő 	  árvöztörő 	 2
      fúró 	        eró 	 2
   ^kacsa$ 	     kacsan 	 2
         a 	         ba 	 1
        és 	         és 	 0

Accuracy on data:  90.0 %
average Levenshtein distance:  1.0


In [5]:
def decode_critical(enc=180):
    for POS in range(enc):
        encoded=np.ones(enc)
        encoded[POS]=0
        a=encoder.decode([encoded])[0].reshape([10,36])

        b=corp.defeaturize_data_charlevel_onehot([a])
        print(b)